### <font color="Cyan">Basic Perceptron</font> after setting up some shit

In [1]:
import tensorflow as tf

def check_tf_device():
    # Check if TensorFlow can see any GPUs
    print("TensorFlow version:", tf.__version__)
    print("\nPhysical devices available:")
    print(tf.config.list_physical_devices())
    
    # More detailed GPU information if available
    gpus = tf.config.list_physical_devices('GPU')
    if gpus:
        print("\nGPU devices found:")
        for gpu in gpus:
            print(f"  {gpu}")
        # Try to create a simple operation on GPU to verify it's working
        with tf.device('/GPU:0'):
            a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
            b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
            c = tf.matmul(a, b)
            print("\nMatrix multiplication result:", c.numpy())
            print("Device placement:", c.device)
    else:
        print("\nNo GPU devices found. TensorFlow is running on CPU.")
        
    # Show current device placement policy
    print("\nDevice placement policy:")
    if tf.config.get_soft_device_placement():
        print("Soft device placement is enabled")
    else:
        print("Soft device placement is disabled")

check_tf_device()

2024-10-06 09:50:21.932490: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-06 09:50:21.990613: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-06 09:50:22.012761: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-06 09:50:22.075970: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-06 09:50:25.131051: W ten

TensorFlow version: 2.17.0

Physical devices available:
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:3', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:4', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:5', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:6', device_type='GPU')]

GPU devices found:
  PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
  PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')
  PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU')
  PhysicalDevice(name='/physical_device:GPU:3', device_type='GPU')
  PhysicalDevice(name='/physical_device:GPU:4', device_type='GPU')
  PhysicalDevice(name='/physical_device:GPU:5', device_type=

2024-10-06 09:50:32.106890: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2021] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 858 MB memory:  -> device: 0, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:88:00.0, compute capability: 8.0
2024-10-06 09:50:32.150138: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2021] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 23703 MB memory:  -> device: 1, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:89:00.0, compute capability: 8.0
2024-10-06 09:50:32.184375: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2021] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 37953 MB memory:  -> device: 2, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:b1:00.0, compute capability: 8.0
2024-10-06 09:50:32.212378: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2021] Created device /job:localhost/replica:0/task:0/device:GPU:3 with 37953 MB memory:  -> device: 3, name: NVIDIA A100-PCIE-40GB, pci bus 


Matrix multiplication result: [[22. 28.]
 [49. 64.]]
Device placement: /job:localhost/replica:0/task:0/device:GPU:0

Device placement policy:
Soft device placement is enabled


In [2]:
!pip install tensorflow
!pip install --upgrade pip

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [3]:
import os
import sys
import subprocess
import tensorflow as tf

def run_command(command):
    try:
        result = subprocess.run(command, shell=True, check=True, capture_output=True, text=True)
        return result.stdout
    except subprocess.CalledProcessError as e:
        return f"Error executing {command}: {e.stderr}"

def check_gpu_prerequisites():
    print("=== TensorFlow GPU Troubleshooting ===\n")
    
    # 1. Check TensorFlow version
    print(f"TensorFlow version: {tf.__version__}")
    
    # 2. Check if CUDA is in PATH
    cuda_path = os.environ.get('CUDA_PATH')
    print(f"\nCUDA_PATH: {cuda_path if cuda_path else 'Not found in environment variables'}")
    
    # 3. Check NVIDIA drivers
    print("\nNVIDIA Driver Info:")
    nvidia_smi = run_command('nvidia-smi')
    print(nvidia_smi if nvidia_smi else "nvidia-smi not found - drivers may not be installed")
    
    # 4. Check CUDA version
    print("\nCUDA Version:")
    nvcc_version = run_command('nvcc --version')
    print(nvcc_version if nvcc_version else "nvcc not found - CUDA toolkit may not be installed")
    
    # 5. Check cuDNN
    cuda_path = os.environ.get('CUDA_PATH')
    if cuda_path:
        cudnn_path = os.path.join(cuda_path, 'include', 'cudnn.h')
        if os.path.exists(cudnn_path):
            print("\ncuDNN found")
        else:
            print("\ncuDNN not found in expected location")
    
    # 6. Check TensorFlow GPU devices
    print("\nTensorFlow GPU Devices:")
    print(tf.config.list_physical_devices('GPU'))
    
    # 7. Print CUDA device environment variables
    print("\nRelevant Environment Variables:")
    cuda_vars = {k: v for k, v in os.environ.items() if 'CUDA' in k}
    for k, v in cuda_vars.items():
        print(f"{k}: {v}")

    # 8. Provide troubleshooting advice
    print("\n=== Troubleshooting Steps ===")
    print("If no GPU is detected, check the following:")
    print("1. Verify NVIDIA drivers are installed and up to date")
    print("2. Ensure CUDA toolkit is installed and matches TensorFlow requirements")
    print("3. Verify cuDNN is installed and matches CUDA version")
    print("4. Check that your TensorFlow version supports your CUDA version")
    print("5. Make sure your GPU is compatible with TensorFlow")
    
    # Print TensorFlow-CUDA compatibility information
    print("\n=== TensorFlow-CUDA Compatibility ===")
    print("TensorFlow 2.10: CUDA 11.2, cuDNN 8.1")
    print("TensorFlow 2.11-2.12: CUDA 11.8, cuDNN 8.6")
    print("TensorFlow 2.13+: CUDA 11.8/12.0, cuDNN 8.6+")


check_gpu_prerequisites()

=== TensorFlow GPU Troubleshooting ===

TensorFlow version: 2.17.0

CUDA_PATH: Not found in environment variables

NVIDIA Driver Info:
Sat Oct  5 21:25:58 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.08              Driver Version: 545.23.08    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-PCIE-32GB           Off | 00000000:14:00.0 Off |                    0 |
| N/A   36C    P0              37W / 250W |   2021MiB / 32768MiB |      0%      Default |
|                      

In [5]:
!python3 -c "import tensorflow as tf; print(tf.reduce_sum(tf.random.normal([1000, 1000])))"

2024-10-05 21:26:02.458271: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-05 21:26:02.490558: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-05 21:26:02.500699: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-05 21:26:02.525788: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-05 21:26:05.466488: W ten

In [6]:
import os
import tensorflow as tf

# Prevent duplicate registrations without disabling optimizations
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

# Import and configure TensorFlow
import tensorflow as tf

# Disable eager execution to potentially improve performance
tf.compat.v1.disable_eager_execution()

# Configure TensorFlow to use memory growth
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

# Optional: Set up mixed precision for potential performance boost
try:
    policy = tf.keras.mixed_precision.Policy('mixed_float16')
    tf.keras.mixed_precision.set_global_policy(policy)
except:
    print("Mixed precision not available, using default precision")

# Your TensorFlow code here
# For example:
print(tf.reduce_sum(tf.random.normal([1000, 1000])))

Physical devices cannot be modified after being initialized
Tensor("Sum:0", shape=(), dtype=float32)


In [2]:


import os
import ctypes
import tensorflow as tf

def force_cuda_registration():
    # Force load CUDA libraries
    ctypes.CDLL("libcudart.so")
    ctypes.CDLL("libcublas.so")
    ctypes.CDLL("libcufft.so")
    ctypes.CDLL("libcurand.so")
    ctypes.CDLL("libcusolver.so")
    ctypes.CDLL("libcusparse.so")
    ctypes.CDLL("libcudnn.so")

    # Set environment variables to force registration





    
    os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
    os.environ['TF_CUDNN_DETERMINISTIC'] = '1'
    os.environ['TF_FORCE_CUDA_ALLOW_GROWTH'] = 'true'

    # Clear any existing GPU devices
    tf.config.experimental.set_memory_growth(tf.config.list_physical_devices('GPU')[0], True)

    # Reinitialize TensorFlow's GPU configuration
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
        try:
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
            logical_gpus = tf.config.experimental.list_logical_devices('GPU')
            print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
        except RuntimeError as e:
            print(e)

# Call the function to force registration
force_cuda_registration()

# Your TensorFlow code here
# For example:
print(tf.reduce_sum(tf.random.normal([1000, 1000])))

OSError: /home/mtech/2021/dibyo/.anaconda/envs/CPU2/lib/python3.12/lib-dynload/../../libcublas.so: undefined symbol: cublasLtHSHMatmulAlgoInit, version libcublasLt.so.11

In [3]:
import tensorflow as tf

print("Is GPU available?", tf.config.list_physical_devices('GPU'))


Is GPU available? [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:3', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:4', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:5', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:6', device_type='GPU')]


In [4]:
import tensorflow as tf
from keras import layers, models
from tensorflow.keras.datasets import mnist

# Load and preprocess the MNIST dataset
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

# Reshape images to include channel dimension
train_images = train_images.reshape((60000, 28, 28, 1))
test_images = test_images.reshape((10000, 28, 28, 1))

# Define the CNN model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(train_images, train_labels, epochs=2, 
                    validation_data=(test_images, test_labels))

# Evaluate the model
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)
print(f'\nTest accuracy: {test_acc}')

/home/mtech/2021/dibyo/.anaconda/envs/CPU2/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/2


I0000 00:00:1728188492.352739 1132287 service.cc:146] XLA service 0x7f81400038e0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1728188492.352767 1132287 service.cc:154]   StreamExecutor device (0): NVIDIA A100-PCIE-40GB, Compute Capability 8.0
I0000 00:00:1728188492.352771 1132287 service.cc:154]   StreamExecutor device (1): NVIDIA A100-PCIE-40GB, Compute Capability 8.0
I0000 00:00:1728188492.352777 1132287 service.cc:154]   StreamExecutor device (2): NVIDIA A100-PCIE-40GB, Compute Capability 8.0
I0000 00:00:1728188492.352782 1132287 service.cc:154]   StreamExecutor device (3): NVIDIA A100-PCIE-40GB, Compute Capability 8.0
I0000 00:00:1728188492.352787 1132287 service.cc:154]   StreamExecutor device (4): Tesla V100-PCIE-32GB, Compute Capability 7.0
I0000 00:00:1728188492.352793 1132287 service.cc:154]   StreamExecutor device (5): Tesla V100-PCIE-32GB, Compute Capability 7.0
I0000 00:00:1728188492.352798 1132287 service.cc:154]   Str

   3/1875 ━━━━━━━━━━━━━━━━━━━━ 1:16 41ms/step - accuracy: 0.0625 - loss: 2.3082  

I0000 00:00:1728188499.455862 1132287 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1875/1875 ━━━━━━━━━━━━━━━━━━━━ 80s 37ms/step - accuracy: 0.9000 - loss: 0.3226 - val_accuracy: 0.9870 - val_loss: 0.0405
Epoch 2/2
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 58s 31ms/step - accuracy: 0.9859 - loss: 0.0468 - val_accuracy: 0.9831 - val_loss: 0.0509
313/313 - 5s - 15ms/step - accuracy: 0.9831 - loss: 0.0509

Test accuracy: 0.9830999970436096


In [1]:
import tensorflow as tf
from keras import layers, models
from tensorflow.keras.datasets import mnist
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load and preprocess the MNIST dataset
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

# Reshape images to include channel dimension
train_images = train_images.reshape((60000, 28, 28, 1))
test_images = test_images.reshape((10000, 28, 28, 1))

# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1
)
datagen.fit(train_images)

# Define the CNN model with additional layers and batch normalization
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),  # Adding dropout for regularization
    layers.Dense(10, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model with data augmentation
history = model.fit(datagen.flow(train_images, train_labels, batch_size=64),
                    epochs=15,  # Increased number of epochs
                    validation_data=(test_images, test_labels))

# Evaluate the model
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)
print(f'\nTest accuracy: {test_acc}')


2024-10-05 21:45:37.833633: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-05 21:45:37.918288: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-05 21:45:37.929373: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-05 21:45:37.989846: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-05 21:45:41.020099: W ten

Epoch 1/15


I0000 00:00:1728144963.171801 1025020 service.cc:146] XLA service 0x7f8378010a20 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1728144963.171847 1025020 service.cc:154]   StreamExecutor device (0): NVIDIA A100-PCIE-40GB, Compute Capability 8.0
I0000 00:00:1728144963.171851 1025020 service.cc:154]   StreamExecutor device (1): NVIDIA A100-PCIE-40GB, Compute Capability 8.0
I0000 00:00:1728144963.171857 1025020 service.cc:154]   StreamExecutor device (2): NVIDIA A100-PCIE-40GB, Compute Capability 8.0
I0000 00:00:1728144963.171861 1025020 service.cc:154]   StreamExecutor device (3): NVIDIA A100-PCIE-40GB, Compute Capability 8.0
I0000 00:00:1728144963.171866 1025020 service.cc:154]   StreamExecutor device (4): Tesla V100-PCIE-32GB, Compute Capability 7.0
I0000 00:00:1728144963.171871 1025020 service.cc:154]   StreamExecutor device (5): Tesla V100-PCIE-32GB, Compute Capability 7.0
I0000 00:00:1728144963.171877 1025020 service.cc:154]   Str

  2/938 ━━━━━━━━━━━━━━━━━━━━ 1:24 91ms/step - accuracy: 0.1758 - loss: 3.1798  

I0000 00:00:1728144978.206789 1025020 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


938/938 ━━━━━━━━━━━━━━━━━━━━ 111s 94ms/step - accuracy: 0.8257 - loss: 0.5591 - val_accuracy: 0.9834 - val_loss: 0.0538
Epoch 2/15
938/938 ━━━━━━━━━━━━━━━━━━━━ 67s 71ms/step - accuracy: 0.9651 - loss: 0.1175 - val_accuracy: 0.9769 - val_loss: 0.0795
Epoch 3/15
938/938 ━━━━━━━━━━━━━━━━━━━━ 71s 76ms/step - accuracy: 0.9752 - loss: 0.0907 - val_accuracy: 0.9813 - val_loss: 0.0690
Epoch 4/15
938/938 ━━━━━━━━━━━━━━━━━━━━ 69s 73ms/step - accuracy: 0.9787 - loss: 0.0753 - val_accuracy: 0.9763 - val_loss: 0.0773
Epoch 5/15
938/938 ━━━━━━━━━━━━━━━━━━━━ 69s 73ms/step - accuracy: 0.9812 - loss: 0.0694 - val_accuracy: 0.9920 - val_loss: 0.0261
Epoch 6/15
938/938 ━━━━━━━━━━━━━━━━━━━━ 69s 73ms/step - accuracy: 0.9811 - loss: 0.0693 - val_accuracy: 0.9909 - val_loss: 0.0315
Epoch 7/15
938/938 ━━━━━━━━━━━━━━━━━━━━ 68s 73ms/step - accuracy: 0.9837 - loss: 0.0599 - val_accuracy: 0.9909 - val_loss: 0.0283
Epoch 8/15
938/938 ━━━━━━━━━━━━━━━━━━━━ 70s 75ms/step - accuracy: 0.9845 - loss: 0.0526 - val_accura

In [21]:
import tensorflow as tf
from tensorflow.python.platform import build_info as tf_build_info

# Check if GPU is available
if tf.config.list_physical_devices('GPU'):
    print("TensorFlow version:", tf.__version__)
else:
    print("No GPU found.")


TensorFlow version: 2.17.0
